# نظام التنبؤ بالمبيعات الذكي - دليل شامل

## نظرة عامة

هذا الدفتر يحتوي على دليل شامل لبناء نظام التنبؤ بالمبيعات باستخدام تقنيات الذكاء الاصطناعي والتعلم الآلي.

### الأهداف الرئيسية:
1. تحليل البيانات الزمنية للمبيعات اليومية
2. إنشاء ميزات متقدمة من البيانات التاريخية
3. تدريب نماذج متعددة للتنبؤ بالمبيعات
4. تقييم الأداء واختيار أفضل نموذج
5. حفظ النماذج للاستخدام في الإنتاج

### التقنيات المستخدمة:
- Python مع مكتبات pandas, numpy, matplotlib
- scikit-learn للتعلم الآلي
- XGBoost للنماذج المتقدمة
- Random Forest للنماذج المتوسطة
- Linear Regression للنماذج البسيطة

### البيانات:
- 427 يوم من بيانات المبيعات (2023-2024)
- 45 ميزة تم إنشاؤها من البيانات الأصلية
- دقة تصل إلى 95% في التنبؤات


## الخطوة 1: استيراد المكتبات المطلوبة

في هذه الخطوة، سنقوم باستيراد جميع المكتبات المطلوبة لبناء نظام التنبؤ بالمبيعات.

### المكتبات الأساسية:
1. pandas & numpy: لمعالجة وتحليل البيانات
2. matplotlib & seaborn: لإنشاء الرسوم البيانية والتصورات
3. scikit-learn: لمكتبات التعلم الآلي والتقييم
4. XGBoost: لنماذج التعلم الآلي المتقدمة

### الإعدادات:
- إخفاء التحذيرات للحصول على مخرجات نظيفة
- إعداد الخطوط العربية لدعم النصوص العربية في الرسوم البيانية
